In [1]:
import pandas as pd
from IPython.display import Markdown

from tulip.core.collection import TulipCollection
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.plots import plot_area, plot_lines
from tulip.genai import iris
from tulip.data.gs import GSClient as gs

import pycountry

from tulip.analysis.country_related.analytics import summarize_gs_eco_fct

ISO_2 = "AU"

pyctry = pycountry.countries.get(alpha_2=ISO_2)
import plotly.offline as pyo

pyo.init_notebook_mode(connected=True)

Haver path setting remains unchanged.



## Australia
### Activity Indicators
#### Economic Forecasts (Brokers)

In [2]:
gs_eco_fct = gs.get_eco_forecast(geographyId=ISO_2)
gs_summary = summarize_gs_eco_fct(gs_eco_fct)
gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
gs_summary.style.set_caption(f"Goldman {pyctry.name} Economic Forecasts").format(
    precision=2
)

metric,core_cpi,cpi_avg,current_account,rgdp_qoq,rgdp_yoy,output_gap
forecastValue,2.90,2.78,-1.85,2.08,1.70,-0.33


#### Current Activity Indicator (Goldman)

In [ ]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ISO_2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{pyctry.name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
    tick_suffix=" %",
    source="GS",
)
cai_plot

#### Melbourne Institute Nowcasts and Leading Growth Indicators
[Latest](https://melbourneinstitute.unimelb.edu.au/__data/assets/pdf_file/0005/5347166/Nowcast_2507.pdf) Melbourne Institute lowers its nowcast of Australian economic growth in the June quarter

[Latest](https://melbourneinstitute.unimelb.edu.au/publications/macroeconomic-reports/latest-news/leading-index) Weekly Indicators Commentary

In [4]:
mi_leading_growth = hc.create_collection(
    {
        "AU WMI Lead Index (NSA, 1996=100)": "AUNEVLDE@ANZ",
        "WMI Lead Index – Mo. Dev. from Trend (% NSA)": "AUNEVLDM@ANZ",
        "WMI Lead Index – 6M Ann. Growth (% NSA)": "AUNEVLAG@ANZ",
        "6M Ann. Growth Contrib. – S&P/ASX 200 (%Pt NSA)": "AUNEVLSP@ANZ",
        "6M Ann. Growth Contrib. – WMI CSI Exp. Idx (%Pt NSA)": "AUNEVLEI@ANZ",
        "6M Ann. Growth Contrib. – US Ind. Prod. (%Pt NSA)": "AUNEVLIP@ANZ",
        "6M Ann. % Contrib. – RBA Comm. Prices AUD (%Pt NSA)": "AUNEVLCP@ANZ",
        "6M Ann. Growth Contrib. – Dwelling App (%Pt NSA)": "AUNEVLDA@ANZ",
        "6M Ann. Growth Contrib. – Yield Spread (%Pt NSA)": "AUNEVLRG@ANZ",
        "6M Ann. Growth Contrib. – Westpac-MI Unemp. Exp Idx (%Pt NSA)": "AUNEVLUE@ANZ",
        "6M Ann. Growth Contrib. – Agg. Mo. Hrs Worked (%Pt NSA)": "AUNEVLHW@ANZ",
    }
)

# mi_leading_growth["AUNEVLUE@ANZ"].good_is = -1 # I think they are all put in the right order

In [5]:
leading_insight = iris.summarize(
    mi_leading_growth,
    "what trends do you see here? Could you also review the text of this page? https://melbourneinstitute.unimelb.edu.au/publications/macroeconomic-reports/latest-news/leading-index",
)
leading_insight.html()

In [6]:
mi_leading_growth.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"AU WMI Lead Index (NSA, 1996=100)",97.2,2025-09-30,97.2,-0.03,-0.14,-0.15,0.25,2025-11-12 19:14:00
WMI Lead Index – Mo. Dev. from Trend (% NSA),-0.03,2025-09-30,-0.08,0.05,0.19,-0.11,-0.04,2025-11-12 19:14:00
WMI Lead Index – 6M Ann. Growth (% NSA),0.04,2025-09-30,-0.24,0.28,0.47,-0.79,0.28,2025-11-12 19:14:00
6M Ann. Growth Contrib. – S&P/ASX 200 (%Pt NSA),0.19,2025-09-30,0.14,0.05,0.25,0.21,0.14,2025-11-12 19:14:00
6M Ann. Growth Contrib. – WMI CSI Exp. Idx (%Pt NSA),-0.04,2025-09-30,-0.05,0.01,0.06,-0.21,-0.15,2025-11-12 19:14:00
6M Ann. Growth Contrib. – US Ind. Prod. (%Pt NSA),0.03,2025-09-30,-0.01,0.04,0.15,-0.02,0.05,2025-11-12 19:14:00
6M Ann. % Contrib. – RBA Comm. Prices AUD (%Pt NSA),-0.12,2025-09-30,-0.20,0.08,0.39,-0.19,0.18,2025-11-12 19:14:00
6M Ann. Growth Contrib. – Dwelling App (%Pt NSA),-0.11,2025-09-30,-0.06,-0.05,-0.27,-0.21,-0.19,2025-11-12 19:14:00
6M Ann. Growth Contrib. – Yield Spread (%Pt NSA),0.14,2025-09-30,0.14,0.00,0.00,-0.09,0.20,2025-11-12 19:14:00
6M Ann. Growth Contrib. – Westpac-MI Unemp. Exp Idx (%Pt NSA),-0.03,2025-09-30,-0.13,0.10,0.49,-0.24,0.02,2025-11-12 19:14:00


In [7]:
mi_leading_growth.dashboard.plots(years_limit=4, show_0=True)

#### Australis nowcast

In [8]:
ifs_nowcast = bb.get_series("BENWAUGC Index")
ifs_nowcast.plot(years_alimit=2, tick_suffix="%")

#### Retail Sales

In [9]:
retail_sales = bb.create_collection(
    {
        "Australia Retail Sales YoY": "AURSTYSA Index",
        "Australia Retail Sales MoM": "AURSTSA  Index",
    }
)
retail_sales.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
Australia Retail Sales YoY,4.9,2025-06-30,3.5,1.4,0.59,2.0,2.1,-,2025-11-12 19:14:00
Australia Retail Sales MoM,1.2,2025-06-30,0.50,0.70,0.27,0.60,1.3,-,2025-11-12 19:14:00


In [10]:
retail_sales[0].plot(years_limit=4, tick_suffix="%", show_0=True)

#### PMIs

In [11]:
pmi = {
    "S&P Manufacturing PMI": f"MPMI{ISO_2}SA Index",
    "S&P Composite PMI": f"MPMI{ISO_2}CA Index",
    "Australia General Industry PMI": "AGIIPMI  Index",
    # "Ivey PMI": "IVEYSA   Index",
}

pmi_collection = bb.create_collection(pmi)
pmi_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
S&P Manufacturing PMI,52.5,2025-10-31,52.4,0.10,0.05,1.5,1.5,-,2025-11-12 19:14:00
S&P Composite PMI,52.1,2025-10-31,52.4,-0.30,-0.17,1.1,1.9,-,2025-11-12 19:14:00
Australia General Industry PMI,-22.0,2025-10-31,-15.7,-6.3,-0.73,4.9,-0.40,-,2025-11-12 19:14:00


In [12]:
pmi_collection.dashboard.plots(hlines=50, show_0=True)

#### Growth Stats
##### GDP Growth

In [13]:
nom_gdp_yoy = hc.get_series("AUSNGDP@ANZ")
nom_gdp_yoy.ts = nom_gdp_yoy.ts.pct_change(4).mul(100)
real_gdp_yoy = hc.get_series("AUSNGDPC@ANZ")
real_gdp_yoy.ts = real_gdp_yoy.ts.pct_change(4).mul(100)
TulipCollection([nom_gdp_yoy, real_gdp_yoy]).dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Australia: Gross Domestic Product (SA, Mil.A$)",4.1,2025-06-30,3.7,0.35,0.16,0.62,-0.32,2025-11-12 19:14:00
"Australia: Gross Domestic Product (SA, Mil.Chn.FY 2023.A$)",1.8,2025-06-30,1.4,0.47,0.26,1.1,0.89,2025-11-12 19:14:00


In [14]:
plot_lines(
    [nom_gdp_yoy.ts.rename("Nominal"), real_gdp_yoy.ts.rename("Real")],
    tick_suffix="%",
    title="<b>Australian GDP YoY</b>",
    years_limit=10,
    show_0=True,
)

##### GDP Detail

In [15]:
level_stats = {
    "Real GDP": "AUSNGDPC@ANZ",  # Real Gross Domestic Product at Market Prices
    "Fixed Investment": "AUSNFC@ANZ",  #  GDP: Gross Fixed Capital Formation
    "Inventories": "AUSNBC@ANZ",  # GDP: Investment in Inventories
    "Consumption": "AUSNCC@ANZ",  # : GDP: Final Consumption Expenditure
    "HH Consumption": "AUSNHC@ANZ",  # GDP: HH Final Consumption Expenditure
    "Gov Consumption": "AUSNGC@ANZ",  # GDP: Gob Final Consumption Expenditure
    "Exports": "AUSNXC@ANZ",  #  GDP: Exports of Goods and Services
    "Imports": "AUSNMC@ANZ",  # GDP: Imports of Goods and Services
}

level_collection = hc.create_collection(level_stats)
level_collection[-1].good_is = -1
level_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Real GDP,"664,211",2025-06-30,"660,243","3,968",0.99,"9,673","11,862",2025-11-12 19:14:00
Fixed Investment,"157,674",2025-06-30,"158,969","-1,295",-0.69,-808.0,"1,591",2025-11-12 19:14:00
Inventories,"1,199",2025-06-30,"2,108",-909.0,-0.44,"1,296",-608.0,2025-11-12 19:14:00
Consumption,"483,402",2025-06-30,"478,943","4,459",1.3,"9,605","12,248",2025-11-12 19:14:00
HH Consumption,"333,168",2025-06-30,"330,219","2,949",0.89,"6,636","6,524",2025-11-12 19:14:00
Gov Consumption,"150,235",2025-06-30,"148,723","1,512",1.8,"2,970","5,725",2025-11-12 19:14:00
Exports,"181,032",2025-06-30,"177,930","3,102",1.4,"2,403","2,625",2025-11-12 19:14:00
Imports,"159,426",2025-06-30,"157,213","2,213",0.96,"3,198","3,047",2025-11-12 19:14:00


In [16]:
growth_stats = {
    "Real GDP": "yryr%(AUSNGDPC@ANZ)",  # Real Gross Domestic Product at Market Prices
    "Fixed Investment": "yryr%(AUSNFC@ANZ )",  #  GDP: Gross Fixed Capital Formation
    "Inventories": "AUSNBC@ANZ",  # GDP: Investment in Inventories
    "Consumption": "yryr%(AUSNCC@ANZ)",  # : GDP: Final Consumption Expenditure
    "HH Consumption": "yryr%(AUSNHC@ANZ)",  # GDP: HH Final Consumption Expenditure
    "Gov Consumption": "yryr%(AUSNGC@ANZ)",  # GDP: Gob Final Consumption Expenditure
    "Exports": "yryr%(AUSNXC@ANZ)",  #  GDP: Exports of Goods and Services
    "Imports": "yryr%(AUSNMC@ANZ)",  # GDP: Imports of Goods and Services
}

growth_stats = hc.create_collection(growth_stats)
growth_stats[-1].good_is = -1
growth_stats.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Real GDP,1.8,2025-06-30,1.4,0.47,0.26,1.1,0.89,2025-11-12 19:14:00
Fixed Investment,1.0,2025-06-30,2.5,-1.5,-0.36,-0.92,-0.36,2025-11-12 19:14:00
Inventories,"1,199",2025-06-30,"2,108",-909.0,-0.44,"1,296",-608.0,2025-11-12 19:14:00
Consumption,2.6,2025-06-30,1.8,0.75,0.48,0.79,1.00,2025-11-12 19:14:00
HH Consumption,2.0,2025-06-30,0.81,1.2,0.59,1.7,1.7,2025-11-12 19:14:00
Gov Consumption,4.0,2025-06-30,4.2,-0.28,-0.10,-1.3,-0.74,2025-11-12 19:14:00
Exports,1.5,2025-06-30,-0.57,2.0,0.32,2.3,1.5,2025-11-12 19:14:00
Imports,1.9,2025-06-30,0.52,1.4,0.21,-1.0,-3.9,2025-11-12 19:14:00


In [17]:
growth_stats.dashboard.plots(show_0=True, years_limit=3, tick_suffix="%")

### Prices

In [18]:
prices = {
    "CPI": "yryr%(AUWPC@ANZ)",
    "Core CPI": "yryr%(AUWPCXH@ANZ)",
    "CPI NSA": "yryr%(AUMPC@ANZ)",
    # "RBA Headline": "yryr%(AUSPCP@ANZ)",
    # "RBA Trimmed mean": "yryr%(AUSPCTRP@ANZ)",
    # "RBA Weighted median": "yryr%(AUSPCWP@ANZ)",
}

prices_collection = hc.create_collection(prices)
prices_collection.good_is = -1
prices_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
CPI,3.5,2025-09-30,3.0,0.49,0.88,1.1,1.2,2025-11-12 19:14:00
Core CPI,3.7,2025-09-30,3.3,0.40,1.00,1.0,0.89,2025-11-12 19:15:00
CPI NSA,3.5,2025-09-30,3.0,0.48,0.83,1.1,1.3,2025-11-12 19:15:00


In [19]:
prices_df = prices_collection.df
prices_df.columns = list(prices.keys())
fig = plot_lines(
    prices_df,
    years_limit=5,
    title="<b>Inflation Measures</b>",
    tick_suffix="%",
)
fig.add_hline(2, line_width=1, annotation_text="Target")  # , annotation="Target")

### Melbourne Inflation Gauge

In [20]:
melbourne_inflation_g = {
    "Melbourne Ins. Monthly Inflation Gauge": "yryr%(AUNIGI@ANZ)",
    "Core excluding volatile items": "yryr%(AUNIGIC@ANZ)",
    "Trimmed mean": "yryr%(AUNIGIT@ANZ)",
}


melbourne_inflation_metrics = hc.create_collection(melbourne_inflation_g)
melbourne_inflation_metrics.good_is = -1
melbourne_inflation_metrics.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Melbourne Ins. Monthly Inflation Gauge,3.1,2025-10-31,3.0,0.06,0.15,-0.25,0.11,2025-11-12 19:15:00
Core excluding volatile items,3.2,2025-10-31,3.2,0.04,0.12,-0.76,-0.52,2025-11-12 19:15:00
Trimmed mean,2.0,2025-10-31,2.0,0.02,0.07,0.09,-0.69,2025-11-12 19:15:00


Melbourne Institute Survey of Consumer Inflationary Expectations (Expected Inflation Rate Over the Next 12 Months)

In [21]:
df = melbourne_inflation_metrics.df
df.columns = melbourne_inflation_g.keys()

fig = plot_lines(
    df,
    years_limit=5,
    title="<b>Melbourne Institution Inflation Gauge Measures</b>",
    tick_suffix="%",
    logo=False,
)
fig.add_hline(2, line_width=1, annotation_text="Target")  # , annotation="Target")

In [22]:
additional_inflation_metrics = {
    "Trimmed mean (headline)": "AUNVIT30@ANZ",
    "Weighted mean (full sample)": "AUNVIW@ANZ",
    "Weighted mean (within 0% to 5% range)": "AUNVIW05@ANZ",
}

add_inflation_surveys = hc.create_collection(additional_inflation_metrics)
add_inflation_surveys.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Trimmed mean (headline),4.5,2025-11-30,4.8,-0.31,-0.60,0.33,0.70,2025-11-12 19:15:00
Weighted mean (full sample),7.0,2025-11-30,5.7,1.3,1.4,1.5,1.5,2025-11-12 19:15:00
Weighted mean (within 0% to 5% range),2.7,2025-11-30,2.8,-0.13,-0.65,0.14,0.32,2025-11-12 19:15:00


In [23]:
df = add_inflation_surveys.df
df.columns = additional_inflation_metrics.keys()

fig = plot_lines(
    df, years_limit=5, title="<b>Inflation Surveys</b>", tick_suffix="%", logo=False
)
fig.add_hline(2, line_width=1, annotation_text="Target")  # , annotation="Target")

<div class="alert alert-block alert-danger">
<b>STOP</b> Canada from here onwards. Still working on it 
</div>

### Business Outlook Surveys

In [24]:
Stop

NameError: name 'Stop' is not defined

In [ ]:
bos_prices = business_outlook_survey_prices.df
bos_prices.columns = list(businaess_outlook_survey_prices_dict.keys())

# Define custom colors for each category
colors = {
    "Above 3 percent": "red",
    "2 to 3 percent": "coral",
    "1 to 2 percent": "lightgreen",
    "Below 1 percent": "green",
    "No response": "gray",
}

fig = plot_area(
    bos_prices,
    title="<b>Business Outlook Survey - Inflation Expectations</b>",
    tick_suffix="%",
    logo=False,
)

for trace in fig.data:
    if trace.name in colors:
        trace.line.color = colors[trace.name]

fig.show()

Other faster, idiosyncratic price data points available in Canada

In [ ]:
weekly_price = {
    "Commodity Price Index": "P156CP@INTWKLY",
    "Commodity Price Index Ex Energy": "P156CPX@INTWKLY",
    "Gasoline Price": "P156GRT@INTWKLY",
    "Pork Prices": "P156HGV@INTWKLY",
}

weekly_price_collection = []
for k, v in weekly_price.items():
    weekly_price_collection.append(hc.get_series(v))

weekly_price_collection = TulipCollection(weekly_price_collection)
weekly_price_collection.dashboard.table()


### Employment

In [ ]:
# Employment metrics
employment = {
    "EI Claims": "E156CEA@INTWKLY",  # Canada: Employment Insurance: Claims Established
    "EI Recipients": "E156ATA@INTWKLY",  # Canada: Employment Insurance: Active Recipients
    "Job Loss Probability": "CANVXJL@CANADA",  # Canada: Consumer Expectations Survey: Prob of Losing Job
}

employment_collection = []
for k, v in employment.items():
    employment_collection.append(hc.get_series(v))

employment_collection = TulipCollection(employment_collection)
employment_collection.good_is = {
    "E156CEA@INTWKLY": -1,  # Lower claims is good
    "E156ATA@INTWKLY": -1,  # Lower recipients is good
    "CANVXJL@CANADA": -1,  # Lower probability of job loss is good
}
employment_collection.dashboard.table()


In [ ]:
employment_collection.dashboard.plots()

### Output Gap

In [ ]:
output_gap = hc.get_series("B156IGAP@CANADA")
output_gap.plot(show_0=True, tick_suffix="%")

In [ ]:
weekly_price_collection.dashboard.plots(years_limit=3)

### Trade

In [ ]:
trade_stats = {
    "Current Account Balance % GDP": "s156bcpg@g10",
}

trade_collection = []
for k, v in trade_stats.items():
    trade_collection.append(hc.get_series(v))

trade_collection = TulipCollection(trade_collection)
trade_collection.dashboard.table()

In [ ]:
trade_collection.dashboard.plots(show_0=True)

### Surveys

In [ ]:
surveys = {
    "Business Barometer 3M": "CFO3@CANADA",  # Canada: Business Barometer Index: 3 Month Outlook
    "Business Barometer 1Y": "CFAA@CANADA",  # Business Barometer Index: Canada
    "Policy Uncertainty": "CEPUIN@CANADA",  # Canada: News-Based Policy Uncertainty Index
}

surveys_collection = []
for k, v in surveys.items():
    surveys_collection.append(hc.get_series(v))

surveys_collection = TulipCollection(surveys_collection)
surveys_collection.good_is = {
    "CFO3@CANADA": 1,  # Higher business confidence is good
    "CFAA@CANADA": 1,  # Higher business confidence is good
    "CEPUIN@CANADA": -1,  # Lower uncertainty is good
}
surveys_collection.dashboard.table()


In [ ]:
surveys_collection.dashboard.plots(years_limit=3, hlines=[50, 50])

### Credit Creation

In [ ]:
debt_stats = {
    "Non-Financial Debt % GDP": "S156zdnp@g10",
    "Household Debt to Disposable Income": "S156zdhi@g10",
    "Household Debt % GDP": "S156zdhp@g10",
}

debt_collection = []
for k, v in debt_stats.items():
    debt_collection.append(hc.get_series(v))

debt_collection = TulipCollection(debt_collection)
debt_collection.dashboard.table()


In [ ]:
debt_collection.dashboard.plots()

### Budget

In [ ]:
government_finance = {
    "Public Debt": "N156FDP@G10",  # Canada: Business Barometer Index: 3 Month Outlook
    "Govt Budget Surplus/Deficit": "H156FGBP@G10",  # Business Barometer Index: Canada
}

government_finance_collection = []
for k, v in government_finance.items():
    government_finance_collection.append(hc.get_series(v))

government_finance_collection = TulipCollection(government_finance_collection)

government_finance_collection.dashboard.table()

In [ ]:
government_finance_collection.dashboard.plots()

### Asset Pricing

In [ ]:
# Asset Piricing

# Yields and Stock

In [ ]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")